In [4]:
import random
from copy import deepcopy

In [28]:
perc_mut = 0.05

selected = []
for i in range(1000000):
	if random.random() < perc_mut:
		selected.append(True)

print((len(selected)/1000000) * 100)

5.0001


In [29]:
len(selected)

50001

In [33]:
ls = ["Hello", "world", "como", "estas"]
ls[ls.index("como")] = "how"



In [5]:
def ls_ls(mut_ls):
	# deepcopy is required to prevent over writing of the original subnetwork list
	# -- mutations are done by index updating
	subnetworks = deepcopy(mut_ls)
	for ls in subnetworks:
		mut_network = []
		for idx, elm in enumerate(ls):
			if idx % 2 == 0:
				ls[ls.index(elm)] = "mutated"

	return subnetworks
		
		

In [6]:
test_ls = [[1,2,3], [4,5,6], [10, 10, 10]]
new_ls = ls_ls(test_ls) 

In [7]:
print(test_ls)

[[1, 2, 3], [4, 5, 6], [10, 10, 10]]


In [8]:
new_ls

[['mutated', 2, 'mutated'],
 ['mutated', 5, 'mutated'],
 ['mutated', 'mutated', 10]]